# Entrenamiento para ejercicio 1

Se etiquetan los videos a partir del nombre etiquetado

In [1]:
import json
import re

input_json_path = "/content/drive/MyDrive/pract3Landmrks/landmarks_pose_hands_ejer1.json"
output_json_path = "/content/drive/MyDrive/pract3Landmrks/labelsejer1.json"

with open(input_json_path, "r") as file:
    data = json.load(file)


pattern = re.compile(r"(full|partial|none)_\d+_\d+")


for video_name, frames in data.items():
    match = pattern.match(video_name)
    if match:
        label = match.group(1)  # Extraer la etiqueta ('full', 'partial' o 'none')


        for frame in frames:
            frame["label"] = label


with open(output_json_path, "w") as file:
    json.dump(data, file, indent=4)

print(f"Etiquetas agregadas y guardadas en {output_json_path}")


Etiquetas agregadas y guardadas en /content/drive/MyDrive/pract3Landmrks/labelsejer1.json


In [ ]:
frame_sizes = [len(frame["pose"]) * 3 + len(frame["hands"]) * 3 for video in data.values() for frame in video]
print("Tamaños de los frames únicos:", set(frame_sizes))

Tamaños de los frames únicos: {12}


In [2]:
import numpy as np

X, y = [], []
expected_pose_points = 3
expected_hand_points = 21

for video_name, frames in data.items():
    for frame in frames:
        landmarks = []


        pose_landmarks = frame["pose"]["left_arm"] + frame["pose"]["right_arm"]
        while len(pose_landmarks) < expected_pose_points:
            pose_landmarks.append([0, 0, 0])
        for point in pose_landmarks:
            landmarks.extend(point)


        hand_landmarks = frame["hands"]
        while len(hand_landmarks) < 2:
            hand_landmarks.append([[0, 0, 0]] * expected_hand_points)  # Mano vacía
        for hand in hand_landmarks:
            for point in hand:
                landmarks.extend(point)

        X.append(landmarks)
        y.append(frame["label"])  #(full, partial, none)

X = np.array(X)
y = np.array(y)

print("Forma de X después de normalización:", X.shape)


Forma de X después de normalización: (7698, 144)


Se normalizan los datos para landmarks faltantes por errores en la extracción

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # full → 0, partial → 1, none → 2


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print("Tamaño de los datos de entrenamiento:", X_train.shape)
print("Tamaño de los datos de prueba:", X_test.shape)

Tamaño de los datos de entrenamiento: (6158, 144)
Tamaño de los datos de prueba: (1540, 144)


## Entrenamiento de modelo RandomForest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Precisión del modelo: 0.98
Reporte de clasificación:
               precision    recall  f1-score   support

        full       0.99      0.97      0.98       592
        none       1.00      1.00      1.00       393
     partial       0.97      0.99      0.98       555

    accuracy                           0.98      1540
   macro avg       0.99      0.99      0.99      1540
weighted avg       0.98      0.98      0.98      1540



Guardo el modelo para test


In [7]:
import pickle
with open("/content/drive/MyDrive/pract3Landmrks/modeloEjer1_rf.pkl", "wb") as f:
    pickle.dump(model, f)

with open("/content/drive/MyDrive/pract3Landmrks/label_encoderEjer1.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("Modelo guardado exitosamente.")

Modelo guardado exitosamente.


In [ ]:


from google.colab import files


uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Video subido: {video_path}")

with open("/content/drive/MyDrive/pract3Landmrks/modeloEjer1_rf.pkl", "rb") as f:
    model = pickle.load(f)

with open("/content/drive/MyDrive/pract3Landmrks/label_encoderEjer1.pkl", "rb") as f:
    label_encoder = pickle.load(f)


with open('/content/drive/MyDrive/pract3Landmrks/nuevo_video.json', 'r') as f:
    nuevo_video_data = json.load(f)

nuevo_features = extract_video_features(nuevo_video_data)
import pandas as pd
nuevo_df = pd.DataFrame([nuevo_features])


prediccion = model.predict(nuevo_df)
clase_predicha = label_encoder.inverse_transform(prediccion)[0]

print(f"El modelo predice que el video pertenece a la clase: {clase_predicha}")
